#### FPGA-ACCELERATION OF MACHINE LEARNING , A CASE STUDY USING A SIMPLE ALTERNATING LEAST SQUARES(ALS) RECOMMENDATION ENGINE.

##### MOVIE RECOMMENDATION USING  MATRIX FACTORIZATION TRAINED WITH ALS
<img src="m_f.png">

This notebook demonstrates, how to communicate to programming-logic(FPGA - pl) from a processing system(ps) running python. The purpose of the communication is to send data to the pl , which has been designed to perform application specific accelerated data computation, and then get the result of the computation back to the ps for further computation. In that way we can achieve massive performance improvements both in speed and and energy consumption. 



### Down below , we give a brief variable explanation, so that it is easier for the reader to understand the source code.

1. numMovies :   **integer**
>  number of Movies

2. numUsers : **integer**
> number of Users

3. M,U : **float arrays**
> M containts one row for each Movie.
> U contains one row for each User.
> the size of each row is NFEATS.
> as a result **U** has size **numUsers x NFEATS** 
> and  **M** has size **numMovies x NFEATS**
> so that **R =  U X M'** has size **numUsers X numMovies** 

4. NFEATS : **integer**
> specifies the factorization size

5. userInf: **array of dictionaries**
> userInf holds the input data in a structured manner sorted by user IDs.
> i.e. userInf[0] = {"iD" : 0 , "numRatings" : 3 , "ratings": [2,3,4] , "rId": [1,2,5]}
> which means that<br>
> **user** with **id = 0** has <br>
> **rated 3 movies** , <br>
> he rated **movie with iD = 1** with **2**, <br>
> he rated **movie with iD = 2** with **3**, <br>
> he rated **movie with iD = 5** with **4** .<br>

6. movieInf: **array of dictionaries**
> movieInf holds the input data in a structured manner sorted by movie IDs.
> i.e. movieInf[0] = {"iD" : 0 , "numRatings" : 3 , "ratings": [1,3,4] , "rId": [4,2,5]}
> which means that <br>
> **movie** with **id = 0** has <br>
> **been rated by 3 users ** ,  <br>
> rated by **user with iD = 4** with **1**, <br>
> rated by **user with iD = 2** with **3**, <br>
> rated by **user with iD = 5** with **4**  <br>

## 1. A function responsible for the Initial hardware configurations

In [1]:
def createHardwareInterface():
    
    #OVERLAY CONFIGURES FPGA USING THE SPECIFIED BITSTREAM
	ol = Overlay("ALS2.bit")
	ol.download()

	DMA_TO_DEV   = 0
	DMA_FROM_DEV = 1
	
	#DMA0 ---> TRANSFERS SBUFFER
	#DMA1 ---> TRANSFERS RBUFFER
	#DMA2 ---> TRANSFERS AOUT
	#DMA3 ---> TRANSFERS VOUT

	#-PHYSICAL ADDRESSES OF THE DMAs AS READ BY THE TCL FILE----
	ADDR_DMA0_BASE = int(PL.ip_dict["SEG_dm_0_Reg"][0],16)
	ADDR_DMA1_BASE = int(PL.ip_dict["SEG_dm_1_Reg"][0],16)
	ADDR_DMA2_BASE = int(PL.ip_dict["SEG_dm_2_Reg"][0],16)
	ADDR_DMA3_BASE = int(PL.ip_dict["SEG_dm_3_Reg"][0],16)
	ADDR_MMIO      = int(PL.ip_dict["SEG_topLevelHW_0_if_Reg"][0],16)
	RANGE_MMIO     = int(PL.ip_dict["SEG_topLevelHW_0_if_Reg"][1],16)
	#------CREATE DMA OBJECTS---------------------------
    
	# direction = 0 send to PL ,  direction = 1 receive from PL
	
	SbufferDMA = DMA(ADDR_DMA0_BASE,direction = DMA_TO_DEV)
	RbufferDMA = DMA(ADDR_DMA1_BASE,direction = DMA_TO_DEV)
	VoutDMA    = DMA(ADDR_DMA2_BASE,direction = DMA_FROM_DEV)
	AoutDMA    = DMA(ADDR_DMA3_BASE,direction = DMA_FROM_DEV)
	mmio 	   = MMIO(ADDR_MMIO, RANGE_MMIO)

	#--------BUFFER CREATION-------------------------------
    #-- THE BUFFERS ARE CREATED AT A MAXIMUM SIZE
    #-- BUT WE WILL STREAM TO HARDWARE  THE DESISED
    #-- SIZE IN EACH ITERATION
	SbufferDMA.create_buf(4*g.NMAXRAT*g.NFEATS,1)
	RbufferDMA.create_buf(4*g.NMAXRAT,1)
	AoutDMA   .create_buf(4*g.NFEATS*g.NFEATS,1)
	VoutDMA   .create_buf(4*g.NFEATS,1)
    # THESE BUFFERS ARE NOTHING MORE THA PHYSICAL CONTINUES ARRAYS
    # OF DATA, SUITABLE FOR DMA TRANSFER
	#-----------------------------------------------------
	# --------get DMAs Buffer Address
	Sbuffer = SbufferDMA.get_buf(32,data_type = 'float')
	Rbuffer = RbufferDMA.get_buf(32,data_type = 'float')
	Aout    = AoutDMA   .get_buf(32,data_type = 'float')
	Vout    = VoutDMA   .get_buf(32,data_type = 'float')

	dmas        = {'SbufferDMA':SbufferDMA ,'RbufferDMA':RbufferDMA ,'AoutDMA': AoutDMA,'VoutDMA': VoutDMA , 'MMIO': mmio }
	dmaBuffers  = {'Sbuffer'   :Sbuffer    ,'Rbuffer'  :Rbuffer    ,'Aout'   :Aout    ,'Vout'   :Vout}
    
	return dmas

## 2. A software - hardware API function, responsible for sending and receiving data from the FPGA

In [2]:
def topLevelHW(nratings,dmas):
    
	SbufferDMA = dmas["SbufferDMA"] 
	RbufferDMA = dmas["RbufferDMA"]
	AoutDMA    = dmas["AoutDMA"]
	VoutDMA    = dmas["VoutDMA"]
	mmio       = dmas["MMIO"]

	DMA_TO_DEV   = 0
	DMA_FROM_DEV = 1

    #WE USED AN IP CALLED "FIFO ACCELERATOR ADAPTER" TO 
    #AVOID DATA STREAMING DIRECTLY TO OUR ACCELERATOR
    #THE FOLLOWING REGISTERS ARE USER TO PROGRAMM THE
    #"FIFO ACCELERATOR ADAPTER"
	CMD_REG           = 0x0028
	CTR_REG           = 0x0000
	ISCALAR0_DATA     = 0x0080
	ISCALAR0_STATUS   = 0x0180
	IARG0_STATUS      = 0x0100
	IARG1_STATUS      = 0x0104
	OARG0_STATUS      = 0x0140
	OARG1_STATUS      = 0x0144

    #sending Scalar nratings to "FIFO ACCELERATOR ADAPTER" , 
    #which specifies the ammount of data to be transfered
    #we use axilite for the transfer
	mmio.write(ISCALAR0_DATA,int(nratings))
	#send command update output
	mmio.write(CMD_REG,0x00010003)
	#send command execute 
	mmio.write(CMD_REG,0x00020000)
	#send command update input
	mmio.write(CMD_REG,0x103)
	
    #SEND INPUT ARRAYS TO HARDWARE
	#transfer(NUM_OF_BYTES_TO_TRANSFER,DIRECTION)
	SbufferDMA.transfer(4*nratings*g.NFEATS,DMA_TO_DEV)
	RbufferDMA.transfer(4*nratings         ,DMA_TO_DEV)
	
	SbufferDMA.wait()
	RbufferDMA.wait()
	
    #--RECEIVE HARDWARE OUTPUT
    #--RECEIVE COMPUTATION RESULTS
	AoutDMA.transfer(4*g.NFEATS*g.NFEATS,DMA_FROM_DEV)
	VoutDMA.transfer(4*g.NFEATS         ,DMA_FROM_DEV)

	VoutDMA.wait()
	AoutDMA.wait()

## 3.1 A function that emulates the hardware calculations in software <br> It is only called in case of pure software calculations (mode = 0)

In [3]:
def softwareKernel(M,sel,ratings,numRatings,dmas):
	Aout = dmas["Aout"]
	Vout = dmas["Vout"]
	for i in range(NFEATS):
		for j in range(i,NFEATS):
			if i<=j:
				result = 0
				for k in range(numRatings):
					temp    = sel[k]
					result += M[temp][i]*M[temp][j]	
				Aout[i*NFEATS+j] = result;
			else:
				Aout[i*NFEATS+j] = 500;
								
	for i in range(NFEATS):
		Aout[i*NFEATS+i] += 0.01*numRatings

	for i in range(NFEATS):
		acc = 0
		for k in range(numRatings):
			acc += M[sel[k]][i]*ratings[k] 
		Vout[i] = acc
	return Aout,Vout


## 3.2 A function that fills the dma buffers with the appropriate data , and then calls topLevelHW <br>  It is only called in case of hardware assisted calculations (mode = 1)

In [4]:

def bufferizeAndSend(M,sel,ratings,nratings,dmas):
	buffIndx = 0
	Sbuffer = dmas["SbufferDMA"].get_buf(32,data_type="float")
	Rbuffer = dmas["RbufferDMA"].get_buf(32,data_type="float")
	for i in range(nratings):
		mov = sel[i]
		for j in range(g.NFEATS):
			Sbuffer[buffIndx] = M[mov][j]
			buffIndx+=1
		Rbuffer[i] = ratings[i]

	hardware.topLevelHW(nratings,dmas)
	Aout = dmas["AoutDMA"].get_buf(32,data_type="float")
	for i in range(g.NFEATS):
		Aout[i*g.NFEATS+i] += 0.01*nratings

def hardwareKernel(M,sel,ratings,numRatings,dmas):
	bufferizeAndSend(M,sel,ratings,numRatings,dmas)
	Aout  = dmas["AoutDMA"].get_buf(32,data_type="float")
	Vout  = dmas["VoutDMA"].get_buf(32,data_type="float")

## 4. A function tha solves a linear system , using Cholesky Decomposition

In [5]:
def choleskySolve(A,B,X):
	L = np.zeros(NFEATS*NFEATS,dtype="float32")
	Y = np.zeros(NFEATS,dtype="float32")
    
    #--DECOMPOSE A = L*L' USING CHOLESKY
	for i in range(NFEATS):
		for j in range(i+1):
			s = 0;
			for k in range(j):
				s += L[i*NFEATS+k]*L[j*NFEATS+k]
			
			temp = A[i*NFEATS+j] if i<=j else A[j*NFEATS+i]
			value = math.sqrt(A[i*NFEATS+i]-s)if i==j else (1.0/L[j*NFEATS+j]*(temp-s))
			L[i*NFEATS+j] = value
	
    #--USE BACKWARD AND FORWARD SUBSTITUTION 
    #--IN ORDER TO SOLVE THE SYSTEM
    #--A*X = B => L*L'*X=B 
    #-- => IF L'*X = Y (1) THEN L*Y=B (2)
    # SOLVE (1) AND (2)
    
	for i in range(NFEATS):
		Y[i] = B[i];
		for j in range(i):
			Y[i]-=L[i*NFEATS+j]*Y[j]
		Y[i] = Y[i]/L[i*NFEATS+i]

	for i in range(NFEATS-1,-1,-1):
		X[i] = Y[i]
		for j in range(i+1,NFEATS):
			X[i]-=X[j]*L[j*NFEATS+i]
		X[i] = X[i]/L[i*NFEATS+i]

## 5. The update functions, they are called alternately

In [6]:
def updateU(U,M,numUsers,userInf,dmas):
	for i in range(numUsers):
		sel           = userInf[i]['rId']
		ratings       = userInf[i]['ratings'] 
		numRatings    = userInf[i]['numRatings']
		iD            = userInf[i]['iD']
		#------------------if mode is HW-----------------------
		if mode == 1:
			hardwareKernel(M,sel,ratings,numRatings,dmas)
			Aout  = dmas["AoutDMA"].get_buf(32,data_type="float")
			Vout  = dmas["VoutDMA"].get_buf(32,data_type="float")
			choleskySolve(Aout,Vout,U[iD])
		#------------------if mode is pure SW-------------------
		if mode == 0:
			softwareKernel(M,sel,ratings,numRatings,dmas)
			Aout  = dmas["Aout"]
			Vout  = dmas["Vout"]
			choleskySolve(Aout,Vout,U[iD])
		#-----------------------------------------------------


def updateM(U,M,numMovies,movieInf,dmas):
	for i in range(numMovies):
		sel           = movieInf[i]['rId'] 
		ratings       = movieInf[i]['ratings']
		numRatings    = movieInf[i]['numRatings']
		iD            = movieInf[i]['iD']
		#------------------if mode is HW-----------------------
		if mode == 1:
			hardwareKernel(U,sel,ratings,numRatings,dmas)
			Aout  = dmas["AoutDMA"].get_buf(32,data_type="float")
			Vout  = dmas["VoutDMA"].get_buf(32,data_type="float")
			choleskySolve(Aout,Vout,M[iD])
		#------------------if mode is pure SW------------------
		if mode == 0:
			softwareKernel(U,sel,ratings,numRatings,dmas)
			Aout  = dmas["Aout"]
			Vout  = dmas["Vout"]
			choleskySolve(Aout,Vout,M[iD])
		#-----------------------------------------------------

## 6. A function to calculate the Round Mean Squared Error (RMSE) 

In [7]:
def calculateRmse(U,M,userInf,numUsers,numData):
	acc = 0
	RMSE =0
	for i in range(numUsers):
		userId     = userInf[i]['iD']
		numRatings = userInf[i]['numRatings']
		movieIds   = userInf[i]['rId']
		ratings    = userInf[i]['ratings']
		for j in range(numRatings):
			movie  = movieIds[j]
			temp = 0.0;
			for k in range(NFEATS):
				temp += M[movie][k]*U[userId][k];

			acc += (temp-ratings[j])*(temp-ratings[j]);
						
	return math.sqrt( (acc/numData) )


## 7. Main 

In [8]:
import random
import numpy as np
import math
import time

###  Some initial variable declaration

In [9]:
global NFEATS   
NFEATS      = 80             #--define factorization size
global NMAXRAT  
NMAXRAT     = 7000           #--defines max Hardware Input Size
global mode         
mode        = 0       #--defines mode 0 = SW | 1 = HW |
datasetFile = "example.dat"  #--definet dataset file
delimiter   = ';'            #--defines the delimeter, that split input data
lamda       = 0.1            #--lambda is a factor for overffiting avoidance 
iterations  = 10            #--defines number of iterations
numMovies = 1                #--defines number of movies
numUsers  = 1                #--defines number of users
numData   = 0                #--total entries in the dataset,
                             #--each entry's format is "user delimiter movie delimiter rating" i.e "1;0;3"
uCol      = 0                
mCol      = 1
rCol      = 2
data      = []
Ru        = []
Rm        = []
U         = []
M         = []


### Read dataset

In [10]:
data = np.loadtxt(datasetFile,dtype='float32',delimiter=delimiter)
numData = data.shape[0]

### Sort a dataset copy sorted by userIDs in Ru and sorted by movieIDs in Rm

In [11]:
Ru     = np.sort(data.view('float32,float32,float32'),order=['f0'],axis=0).view(np.float32) 
Rm     = np.sort(data.view('float32,float32,float32'),order=['f1'],axis=0).view(np.float32) 

### Count users and movies

In [12]:
for i in range(1,len(Ru)):
	if Ru[i][uCol]!=Ru[i-1][uCol]:
		numUsers += 1
	if Rm[i][mCol]!=Rm[i-1][mCol]:
		numMovies+= 1

print("numUsers = ",numUsers)
print("numMovies = ",numMovies)

numUsers =  4
numMovies =  5


### Initialize our models parameters to Random Values

In [13]:
U = np.random.rand(numUsers,NFEATS )
M = np.random.rand(numMovies,NFEATS)


### Initialize and create userInf from raw data as mentioned in "Variable Explanation" section

In [14]:
userInf    = []; 
numRatings = 0;	
iD         = 0;

for i in range(numData+1):
		if i!=numData and Ru[i][uCol]==iD:
			numRatings += 1;
		else:
			ratings = []
			rId     = []
			for j in range(numRatings):
				ratings.append(Ru[i-numRatings+j][rCol])
				rId.append(    Ru[i-numRatings+j][mCol])

			userInf.append({"iD":iD,'numRatings':numRatings,'ratings':np.array(ratings,dtype="float32"),'rId':np.array(rId,dtype="int32")})
			iD        += 1
			numRatings = 1

del Ru

### Initialize and create movieInf from raw data as mentioned in "Variable Explanation" section

In [15]:
movieInf   = []
numRatings = 0;
iD         = 0;

for i in range(numData+1):
	if i!=numData and Rm[i][mCol]==iD:
		numRatings += 1
	else:
		ratings = []
		rId     = []
		for j in range(numRatings):
			ratings.append(Rm[i-numRatings+j][rCol])
			rId.append(    Rm[i-numRatings+j][uCol])
			
		movieInf.append({'iD':iD,'numRatings':numRatings,'ratings':np.array(ratings,dtype="float32"),'rId':np.array(rId,dtype="int32")})
		iD        += 1
		numRatings = 1

del Rm

### Initialize the kernel's IO data, 
1. in case of **software (mode = 0)** we use simple numpy arrays
2. in case of **hardware assisted computations (mode = 1)** we use physical continous allocated buffers

In [16]:
if mode == 1:
	print("mode =",g.mode)
	dmas = createHardwareInterface()
else:
	Aout    = np.zeros(NFEATS*NFEATS ,dtype="float32")
	Vout    = np.zeros(NFEATS        ,dtype="float32")
	Sbuffer = np.zeros(NFEATS*NMAXRAT,dtype="float32")
	Rbuffer = np.zeros(NMAXRAT       ,dtype="float32")
	dmas = {"Aout" : Aout , "Vout" : Vout , "Sbuffer" : Sbuffer , "Rbuffer" : Rbuffer }

### start training by alternate calling updateU and updateM

In [17]:
print("RMSE = ",calculateRmse(U,M,userInf,numUsers,numData))

start_time = time.time()

for i in range(iterations):
	print("#-#-#-#-#-ITERATION: ",i, "-#-#-#-#-#")
	updateU(U,M,numUsers ,userInf ,dmas)
	updateM(U,M,numMovies,movieInf,dmas)
	print("RMSE = ",calculateRmse(U,M,userInf,numUsers,numData))

train_time = time.time()-start_time
print("total train time = ",train_time)


RMSE =  17.97780382872786
#-#-#-#-#-ITERATION:  0 -#-#-#-#-#
RMSE =  0.09229149812219402
#-#-#-#-#-ITERATION:  1 -#-#-#-#-#
RMSE =  0.08140989050327772
#-#-#-#-#-ITERATION:  2 -#-#-#-#-#
RMSE =  0.06967116928966603
#-#-#-#-#-ITERATION:  3 -#-#-#-#-#
RMSE =  0.060597921948845716
#-#-#-#-#-ITERATION:  4 -#-#-#-#-#
RMSE =  0.05475477957623897
#-#-#-#-#-ITERATION:  5 -#-#-#-#-#
RMSE =  0.050937963447516325
#-#-#-#-#-ITERATION:  6 -#-#-#-#-#
RMSE =  0.048313296520533906
#-#-#-#-#-ITERATION:  7 -#-#-#-#-#
RMSE =  0.04640206352126649
#-#-#-#-#-ITERATION:  8 -#-#-#-#-#
RMSE =  0.04489719436980095
#-#-#-#-#-ITERATION:  9 -#-#-#-#-#
RMSE =  0.043620870676756046
total train time =  8.974894046783447


In [19]:
someRandomUsers = np.random.choice(numUsers, 2,replace = False)
print(someRandomUsers)
for u in someRandomUsers:
        sel           = userInf[u]['rId']
        ratings       = userInf[u]['ratings'] 
        numRatings    = userInf[u]['numRatings']
        iD            = userInf[u]['iD']
        for i in range(numRatings):
            movie = sel[i]
            prediction = 0
            for j in range(NFEATS):
                prediction += U[u][j]*M[movie][j]
            print("User ",iD," rated Movie ",sel[i]," with ",ratings[i]," we predict ",prediction )

[0 1]
User  0  rated Movie  0  with  5.0  we predict  4.92534270843
User  0  rated Movie  3  with  3.0  we predict  2.96505139748
User  0  rated Movie  4  with  4.0  we predict  3.94396143647
User  1  rated Movie  1  with  4.0  we predict  3.94468361008
User  1  rated Movie  3  with  1.0  we predict  1.01015265686
User  1  rated Movie  4  with  3.5  we predict  3.46601399214
